# Clean Data Load

Loading Qualified Features (Level 1)

In [52]:
import pandas as pd


In [53]:
clean_data = pd.read_parquet('data_output/clean_l1.parquet')

In [54]:
clean_data.head()

,price_paid,deed_date,property_type,town,district,county,combined_address_x,fuzzy_match,match_confidence,BUILDING_REFERENCE_NUMBER,...,MAINHEAT_ENERGY_EFF,MAINHEATCONT_DESCRIPTION,MAINHEATC_ENERGY_EFF,LIGHTING_DESCRIPTION,LIGHTING_ENERGY_EFF,MECHANICAL_VENTILATION,CONSTRUCTION_AGE_BAND,TENURE,combined_address_y,_merge
0,582000,2022-06-06,S,HIGH WYCOMBE,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,...,Good,"Programmer, room thermostat and TRVs",Good,Low energy lighting in 73% of fixed outlets,Very Good,natural,England and Wales: 1900-1929,owner-occupied,49 amersham road hp13 5aa,both
1,582000,2022-06-06,S,HIGH WYCOMBE,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,...,Good,"Programmer, room thermostat and TRVs",Good,Low energy lighting in 53% of fixed outlets,Good,natural,England and Wales: 1900-1929,owner-occupied,49 amersham road hp13 5aa,both
2,582000,2022-06-06,S,HIGH WYCOMBE,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,...,Good,"Programmer, room thermostat and TRVs",Good,No low energy lighting,Very Poor,natural,England and Wales: 1900-1929,owner-occupied,49 amersham road hp13 5aa,both
3,582000,2022-06-06,S,HIGH WYCOMBE,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,...,Good,"Programmer, room thermostat and TRVs",Good,No low energy lighting,Very Poor,natural,England and Wales: 1900-1929,owner-occupied,49 amersham road hp13 5aa,both
4,655000,2024-12-12,S,HIGH WYCOMBE,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,67 amersham road nan hp13 5aa,67a amersham road hp13 5aa,90.909091,1.000529e+10,...,Good,"Programmer, room thermostat and TRVs",Good,Low energy lighting in all fixed outlets,Very Good,natural,England and Wales: 1930-1949,Owner-occupied,67a amersham road hp13 5aa,both


In [55]:
for col in clean_data.columns:
    print(col, clean_data[col].dtype)

price_paid int64
deed_date datetime64[ns]
property_type object
town object
district object
county object
combined_address_x object
fuzzy_match object
match_confidence float64
BUILDING_REFERENCE_NUMBER float64
CURRENT_ENERGY_RATING object
POTENTIAL_ENERGY_RATING object
CURRENT_ENERGY_EFFICIENCY float64
POTENTIAL_ENERGY_EFFICIENCY float64
PROPERTY_TYPE object
BUILT_FORM object
INSPECTION_DATE datetime64[ns]
LOCAL_AUTHORITY object
CONSTITUENCY object
ENVIRONMENT_IMPACT_CURRENT float64
ENVIRONMENT_IMPACT_POTENTIAL float64
ENERGY_CONSUMPTION_CURRENT float64
ENERGY_CONSUMPTION_POTENTIAL float64
CO2_EMISSIONS_CURRENT float64
CO2_EMISS_CURR_PER_FLOOR_AREA float64
CO2_EMISSIONS_POTENTIAL float64
LIGHTING_COST_CURRENT float64
LIGHTING_COST_POTENTIAL float64
HEATING_COST_CURRENT float64
HEATING_COST_POTENTIAL float64
HOT_WATER_COST_CURRENT float64
HOT_WATER_COST_POTENTIAL float64
TOTAL_FLOOR_AREA float64
ENERGY_TARIFF object
MAINS_GAS_FLAG object
MULTI_GLAZE_PROPORTION float64
GLAZED_AREA object


In [56]:
clean_data['HOT_WATER_ENERGY_EFF'].value_counts()

HOT_WATER_ENERGY_EFF
Good             21103
Average           4248
Poor              1785
Very Good         1210
Very Poor          803
Not Available       19
Name: count, dtype: int64

In [57]:
# All the numeric and datetime features
core_features = list(clean_data.select_dtypes(include=['int64','float64', 'datetime64[ns]']).columns)

#Optional features to consider with experimentation
backup_str_features = [
    'county',
    'CONSTRUCTION_AGE_BAND' # worth exploring!
]

# String Features needing to be encoded
encode_str_features = [
    'WINDOWS_ENERGY_EFF',
    'HOT_WATER_ENV_EFF',
    'MAINHEAT_ENERGY_EFF',
    'WALLS_ENERGY_EFF',
    'POTENTIAL_ENERGY_RATING',
    'MAINS_GAS_FLAG',
    'MAINHEATC_ENERGY_EFF',
    'district', 
    'BUILT_FORM',
    'TENURE', # Combine Duplicate Values 
    'WINDOWS_ENV_EFF',
    'PROPERTY_TYPE',
    'GLAZED_AREA', # Combine Duplicate Values
    'MECHANICAL_VENTILATION', # Clean first
    'LIGHTING_ENERGY_EFF', 
    'CURRENT_ENERGY_RATING',
    'ROOF_ENERGY_EFF',
    'WALLS_ENV_EFF',
    'ENERGY_TARIFF',
    'HOT_WATER_ENERGY_EFF'
]

# AI Features for agentic solutions
ai_features = [
    'WALLS_DESCRIPTION',
    'MAINHEAT_DESCRIPTION', # Could be Traditional vs Other for traditional feature
    'HOTWATER_DESCRIPTION', # Could be Mains vs Other as a traditional feature
    'LIGHTING_DESCRIPTION', 
    'ROOF_DESCRIPTION',
    'MAINHEATCONT_DESCRIPTION',
    'FLOOR_DESCRIPTION'
]

# Features to be removed due to redundant information provision
redundant_features = [
    'town',
    'LOCAL_AUTHORITY', # Removing as district accounts for this and makes more sense
    'WINDOWS_DESCRIPTION', # Removed as GLAZED_AREA provides better insight
    '_merge',
    'property_type', #BUILT_FORM is much more useful
    'CONSTITUENCY', # Removed as distric accounts for this 
]

# Id features to retain
id_features = [
    'fuzzy_match',
    'combined_address_y',
    'combined_address_x',
    'match_confidence'
]

In [58]:
all_cols = list(set(clean_data.columns) - set(core_features)-set(backup_str_features)-set(encode_str_features) - set(ai_features) - set(redundant_features) - set(id_features))
if all_cols == []:
    print(f'All columns are accounted for in the feature split.')

All columns are accounted for in the feature split.


In [59]:
clean_data.drop(columns=redundant_features,inplace=True)

Cleaning columns with unclean categorical data before encoding.

In [60]:
mapping = {
    'owner-occupied' : 'Owner Occupied',
    'Owner-occupied' : 'Owner Occupied',
    'rental (private)': 'Private Rental',
    'Rented (private)': 'Private Rental',
    'unknown' : 'Not Available',
    'rental (social)' : 'Social Rental',
    'NO DATA!' : 'Not Available',
    'Rented (social)' : 'Social Rental'
}

clean_data['TENURE'] = clean_data['TENURE'].replace(mapping)

In [61]:
mapping_2 = {
    'NO DATA!' : 'Not Available'
}

clean_data['GLAZED_AREA'] = clean_data['GLAZED_AREA'].replace(mapping_2)

In [62]:
mapping_3 = {
    'NO DATA!' : 'Not Available',
    'mechanical, extract only': 'Other',
    'mechanical, supply and extract':'Other',
    'mechanical ventilation with heat recovery (MVHR)':'Other',
    'positive input from loft':'Other'
}

clean_data['MECHANICAL_VENTILATION'] = clean_data['MECHANICAL_VENTILATION'].replace(mapping_3)

In [63]:
# Renaming feature names with identifier

# Core Feature Prefix: core_
for col in core_features:
    clean_data.rename(columns={col:f'CORE_{col}'}, inplace=True)

# Adding the prefix 1HE to the columns that will need encoding
for col in encode_str_features:
    clean_data.rename(columns={col:f'1HE_{col}'}, inplace=True)

# Backup Features
for col in backup_str_features:
    clean_data.rename(columns={col:f'BACKUP_{col}'}, inplace=True)
    
# AI Feature Prefix: AI_
for col in ai_features:
    clean_data.rename(columns={col:f'AI_{col}'}, inplace=True)

# ID Feature Prefix: ID_
for col in id_features:
    clean_data.rename(columns={col:f'ID_{col}'}, inplace=True)    


In [64]:
for col in clean_data.columns:
    print(col)

CORE_price_paid
CORE_deed_date
1HE_district
BACKUP_county
ID_combined_address_x
ID_fuzzy_match
CORE_match_confidence
CORE_BUILDING_REFERENCE_NUMBER
1HE_CURRENT_ENERGY_RATING
1HE_POTENTIAL_ENERGY_RATING
CORE_CURRENT_ENERGY_EFFICIENCY
CORE_POTENTIAL_ENERGY_EFFICIENCY
1HE_PROPERTY_TYPE
1HE_BUILT_FORM
CORE_INSPECTION_DATE
CORE_ENVIRONMENT_IMPACT_CURRENT
CORE_ENVIRONMENT_IMPACT_POTENTIAL
CORE_ENERGY_CONSUMPTION_CURRENT
CORE_ENERGY_CONSUMPTION_POTENTIAL
CORE_CO2_EMISSIONS_CURRENT
CORE_CO2_EMISS_CURR_PER_FLOOR_AREA
CORE_CO2_EMISSIONS_POTENTIAL
CORE_LIGHTING_COST_CURRENT
CORE_LIGHTING_COST_POTENTIAL
CORE_HEATING_COST_CURRENT
CORE_HEATING_COST_POTENTIAL
CORE_HOT_WATER_COST_CURRENT
CORE_HOT_WATER_COST_POTENTIAL
CORE_TOTAL_FLOOR_AREA
1HE_ENERGY_TARIFF
1HE_MAINS_GAS_FLAG
CORE_MULTI_GLAZE_PROPORTION
1HE_GLAZED_AREA
CORE_EXTENSION_COUNT
CORE_NUMBER_HABITABLE_ROOMS
CORE_NUMBER_HEATED_ROOMS
CORE_LOW_ENERGY_LIGHTING
CORE_NUMBER_OPEN_FIREPLACES
AI_HOTWATER_DESCRIPTION
1HE_HOT_WATER_ENERGY_EFF
1HE_HOT_WA

In [65]:
clean_data.head()

,CORE_price_paid,CORE_deed_date,1HE_district,BACKUP_county,ID_combined_address_x,ID_fuzzy_match,CORE_match_confidence,CORE_BUILDING_REFERENCE_NUMBER,1HE_CURRENT_ENERGY_RATING,1HE_POTENTIAL_ENERGY_RATING,...,AI_MAINHEAT_DESCRIPTION,1HE_MAINHEAT_ENERGY_EFF,AI_MAINHEATCONT_DESCRIPTION,1HE_MAINHEATC_ENERGY_EFF,AI_LIGHTING_DESCRIPTION,1HE_LIGHTING_ENERGY_EFF,1HE_MECHANICAL_VENTILATION,BACKUP_CONSTRUCTION_AGE_BAND,1HE_TENURE,ID_combined_address_y
0,582000,2022-06-06,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,D,B,...,"Boiler and radiators, mains gas",Good,"Programmer, room thermostat and TRVs",Good,Low energy lighting in 73% of fixed outlets,Very Good,natural,England and Wales: 1900-1929,Owner Occupied,49 amersham road hp13 5aa
1,582000,2022-06-06,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,D,B,...,"Boiler and radiators, mains gas",Good,"Programmer, room thermostat and TRVs",Good,Low energy lighting in 53% of fixed outlets,Good,natural,England and Wales: 1900-1929,Owner Occupied,49 amersham road hp13 5aa
2,582000,2022-06-06,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,E,E,...,"Boiler and radiators, mains gas",Good,"Programmer, room thermostat and TRVs",Good,No low energy lighting,Very Poor,natural,England and Wales: 1900-1929,Owner Occupied,49 amersham road hp13 5aa
3,582000,2022-06-06,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,49 amersham road nan hp13 5aa,49 amersham road hp13 5aa,92.592593,5.175796e+09,D,D,...,"Boiler and radiators, mains gas",Good,"Programmer, room thermostat and TRVs",Good,No low energy lighting,Very Poor,natural,England and Wales: 1900-1929,Owner Occupied,49 amersham road hp13 5aa
4,655000,2024-12-12,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,67 amersham road nan hp13 5aa,67a amersham road hp13 5aa,90.909091,1.000529e+10,D,C,...,"Boiler and radiators, mains gas",Good,"Programmer, room thermostat and TRVs",Good,Low energy lighting in all fixed outlets,Very Good,natural,England and Wales: 1930-1949,Owner Occupied,67a amersham road hp13 5aa


In [66]:
for district in clean_data['1HE_district'].value_counts().index:
    clean_data.loc[(clean_data['1HE_PROPERTY_TYPE'] == 'House') & (clean_data['1HE_district'] == district),
               ['1HE_district',
                '1HE_PROPERTY_TYPE',
                '1HE_BUILT_FORM',
                'CORE_TOTAL_FLOOR_AREA',
                'CORE_EXTENSION_COUNT',
                'CORE_NUMBER_HABITABLE_ROOMS',
                'CORE_deed_date',
                'CORE_INSPECTION_DATE',
                'CORE_price_paid']].to_parquet(f'data_output/district_splits/{district}.parquet')
    print(f'Parquet File for {district} saved.')

Parquet File for BUCKINGHAMSHIRE saved.
Parquet File for WINDSOR AND MAIDENHEAD saved.
Parquet File for SLOUGH saved.
Parquet File for BRACKNELL FOREST saved.
Parquet File for SOUTH BUCKS saved.
Parquet File for WYCOMBE saved.
Parquet File for MILTON KEYNES saved.
Parquet File for SOUTH GLOUCESTERSHIRE saved.
Parquet File for CHILTERN saved.
Parquet File for THREE RIVERS saved.


In [ ]:
clean_data.loc[(clean_data['1HE_PROPERTY_TYPE'] == 'House'),
               ['1HE_district',
                '1HE_PROPERTY_TYPE',
                '1HE_BUILT_FORM',
                'CORE_TOTAL_FLOOR_AREA',
                'CORE_EXTENSION_COUNT',
                'CORE_NUMBER_HABITABLE_ROOMS',
                'CORE_deed_date',
                'CORE_INSPECTION_DATE',
                'CORE_price_paid']].to_parquet('data_output/core_features.parquet')

In [70]:
clean_data.loc[(clean_data['1HE_PROPERTY_TYPE'] == 'House')].to_parquet('data_output/clean_features.parquet')